In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [2]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = (G/KBTV) * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = 0)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
phi = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx)
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    elif form == "EDP":
        BF += alpha * Gel_energy_EDP(F).Compile() * dx
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    






In [ ]:
""" WORKING PARAMETERS FOR SIMPLEST BONDED TEST"""
### 1 PHI = -Z
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))

### 2

In [3]:
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 20
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_delta:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        print("Error",Integrate((uk-gfu.components[0])**2,mesh))

    Draw(gfu.components[0],mesh)



PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -8.389407956753942e-07
Solving nonlinear subproblem
Tol achieved -4.1110676994783745e-07
Solving nonlinear subproblem
Tol achieved -2.6840703012825247e-07
Solving nonlinear subproblem
Tol achieved -5.916239294372882e-07
Solving nonlinear subproblem
Tol achieved -4.307580939211227e-07
QN tol achieved 3.905109039905732e-08
Error 34.958960581165854
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 5.861652609229235e-07
Solving nonlinear subproblem
Tol achieved -5.565991385347827e-07
Solving nonlinear subproblem
Tol achieved -6.135931304949228e-07
Solving nonlinear subproblem
Tol achieved -8.058392095305844e-07
QN tol achieved 4.92953692446869e-08
Error 13.547260788898312
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.990177832723068e-07
Solving nonlinear subproblem
Tol achieved -2.6248532323144825e-07
Solving nonlinear subproblem
Tol achieved -6.53653552375855e-07
Solving nonlinear subproblem
Tol

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 2.975421123336904e-07
Solving nonlinear subproblem
Tol achieved -2.790302483548753e-07
QN tol achieved 1.4137852251944087e-08
Error 0.006450899739929371
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 6.64743002200894e-07
Solving nonlinear subproblem
Tol achieved -4.6490342134933704e-07
QN tol achieved 5.1795332653491386e-08
Error 0.0011937098131610722
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 2.5916540723117247e-07
Solving nonlinear subproblem
Tol achieved -5.784996927697035e-07
QN tol achieved 2.7092834607169042e-08
Error 0.0012967902417537526
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 8.901817818711964e-07
Solving nonlinear subproblem
Tol achieved -3.3964746579604634e-07
QN tol achieved 3.054461071482757e-08
Error 0.0007179177688766671
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 3.211525985333351e-07
Solving nonlinear subproblem
Tol achieved -3.02

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 4.707685647496321e-07
Solving nonlinear subproblem
Tol achieved -4.40251954621508e-07
QN tol achieved 1.4400436335041923e-08
Error 0.007105752468030445
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 2.6056358028166065e-07
Solving nonlinear subproblem
Tol achieved -7.279778104836153e-07
QN tol achieved 2.8796841774471174e-08
Error 0.0012848491310583088
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.064592463841264e-07
Solving nonlinear subproblem
Tol achieved -9.086875814847775e-07
QN tol achieved 2.889458562035449e-08
Error 0.0014295051163390076
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.5263196886142583e-07
Solving nonlinear subproblem
Tol achieved -5.440901178701945e-07
QN tol achieved 1.6458428853984188e-08
Error 0.0008177995119453503
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 5.210079272649021e-07
Solving nonlinear subproblem
Tol achieved -4.961

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.36261882701212e-07
Solving nonlinear subproblem
Tol achieved -3.95918388510315e-07
QN tol achieved 6.79584337427579e-08
Error 18.108234342170945
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 2.539109567537744e-07
Solving nonlinear subproblem
Tol achieved -7.349432770168014e-07
QN tol achieved 2.2040497650931597e-07
Error 0.11248814227339483
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 5.279651971944496e-07
Solving nonlinear subproblem
Tol achieved -4.3553218910769313e-07
QN tol achieved 3.753178915197146e-07
Error 0.17648133589635942
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.3630551450296076e-07
Solving nonlinear subproblem
Tol achieved -2.7604812969120904e-07
Solving nonlinear subproblem
Tol achieved -2.566022061858358e-07
QN tol achieved 1.173159931945838e-09
Error 0.3723086310458913
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 7.730266738175551

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.824783582743107e-07
Solving nonlinear subproblem
Tol achieved -6.63615477180165e-07
QN tol achieved 1.6132199852218855e-07
Error 154.13384378191049
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 4.897648252484421e-07
Solving nonlinear subproblem
Tol achieved -3.7928697696681033e-07
QN tol achieved 9.569563625532853e-07
Error 0.3344755015386156
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 2.828478285333035e-07
Solving nonlinear subproblem
Tol achieved -9.157224271711393e-07
Solving nonlinear subproblem
Tol achieved -6.635951126268875e-07
QN tol achieved 1.5951789280314326e-09
Error 0.5406058204239932
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 7.111547882994032e-07
Solving nonlinear subproblem
Tol achieved -6.062068347638717e-07
Solving nonlinear subproblem
Tol achieved -4.1260900463725595e-07
QN tol achieved 1.3543880276289244e-09
Error 1.0387893422432053
PG it: 5
Start

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…